In [51]:
import pandas as pd
import numpy as np
import pickle as p

data_all = pd.read_csv('data_all.csv')

# Test with Subset of Data
np.random.seed(4242)
item_subset = list(np.random.choice(data_all['Item_ID'].unique(), size=50, replace=False))
data_subset = data_all.loc[data_all['Item_ID'].isin(item_subset)].copy()





import requests
import json

url = 'https://fillet.azurewebsites.net/train/'

data = data_subset.to_json()

payload = {'data': data,
           'cv_acc': False,
           'project_id': 8
           }

headers = {'content-type': 'application/json',
           'Accept-Charset': 'UTF-8'
          }

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers, timeout=1200)

In [52]:
r.status_code

504

In [53]:
r.json()['result']

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# CV Accuracies Returned
cv_accuracies = pd.DataFrame.from_dict((r.json()['cv_acc']))
cv_accuracies

In [54]:
# Sample Actual Price 
from helper_functions import *

sales_data = data_subset

sales_data = optimize_memory(sales_data)

sales_data_wide = sales_data.set_index(
    ['Wk', 'Tier', 'Store', 'Item_ID']).unstack(level=-1).reset_index().copy()
sales_data_wide.columns = [
    ''.join(str(i) for i in col).strip()
    for col in sales_data_wide.columns.values
]
sales_data_wide = sales_data_wide.sort_values(
    ['Tier', 'Store', 'Wk'], ascending=True).reset_index(drop=True)

sales_data_wide_clean = sales_data_wide.dropna(axis=0).copy()
dataset = sales_data_wide_clean
price_columns = [
    col for col in sales_data_wide_clean.columns if col.startswith('Price')
]

a = dataset.sample(1)
a_dict = a[price_columns].iloc[0].to_dict()
a_input = {}
for key in a_dict.keys():
    a_input[key.split('_')[1]]=round(a_dict[key],2)

print('Input Prices Quantities:')
a_input

Input Prices Quantities:


{'102': 10.6,
 '197': 10.85,
 '201': 11.3,
 '202': 1.75,
 '203': 5.25,
 '281': 5.6,
 '350': 2.7,
 '1002': 2.75,
 '1152': 4.0,
 '1203': 2.95,
 '1204': 3.6,
 '1502': 3.6,
 '1503': 3.95,
 '1504': 4.55,
 '1615': 2.95,
 '2259': 13.2,
 '2313': 9.15,
 '2511': 3.2,
 '2541': 6.55,
 '2589': 11.8,
 '2736': 10.15,
 '3045': 2.25,
 '3046': 2.9,
 '3047': 3.35,
 '3726': 10.15,
 '3727': 10.85,
 '3728': 11.55,
 '3732': 11.95,
 '3736': 8.4,
 '3738': 9.8,
 '3772': 9.9,
 '4272': 12.45,
 '4411': 5.55,
 '4728': 13.9,
 '4975': 8.05,
 '4978': 13.15,
 '5501': 3.35,
 '5581': 4.15,
 '5662': 0.7,
 '5690': 13.55,
 '6750': 3.4,
 '7109': 4.25,
 '7115': 4.7,
 '7128': 3.8,
 '7129': 4.6,
 '7158': 5.2,
 '7575': 3.7,
 '7967': 5.15,
 '16001': 11.8,
 '16003': 13.2}

In [58]:
# Predict Sales Qty and Compare to Actual

import requests
import json

url = 'https://fillet.azurewebsites.net/predict/'

payload = {'prices': a_input, 'project_id': 8}

headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}

payload = json.dumps(payload)
r = requests.post(url, data=payload, headers=headers)
qty = r.json()['qty_estimates']
qty_outp = {}
for key in qty.keys():
    qty_outp[key] = round(qty[key], 0)
actual = a[[
    col for col in sales_data_wide_clean.columns if col.startswith('Qty_')
]].iloc[0].to_dict()
pd.DataFrame([qty_outp, actual], index=['Estimated', 'Actual']).transpose()

,Estimated,Actual
Qty_1002,3676.0,2793.0
Qty_102,241.0,250.0
Qty_1152,134.0,168.0
Qty_1203,182.0,145.0
Qty_1204,300.0,272.0
Qty_1502,257.0,279.0
Qty_1503,411.0,459.0
Qty_1504,682.0,837.0
Qty_16001,7.0,7.0
Qty_16003,35.0,62.0
